Importing districts.csv file into a dataframe and creating a list of dates between 15-03-2020 and 14-08-2021

In [1]:
import pandas as pd
dist_data = pd.read_csv('districts.csv')
date_lst=pd.date_range(start="15-03-2020",end="14-08-2021").to_pydatetime().tolist()
for i in range(len(date_lst)):
    date_lst[i]=date_lst[i].strftime("%Y-%m-%d")

Importing neighbor-districts-modified.json file and extracting district keys from it

In [2]:
import json
with open('neighbor-districts-modified.json') as json_file:
    data = json.load(json_file)
dist_keys = list(data.keys())

Appending extra key for Delhi as we don't exclusively need in the cowin vaccination data. So, just appending here as it is only merged in districts.csv file

In [3]:
dist_keys.insert(137,"DL_Delhi")

Getting district names from the district keys to match with the District column of districts.csv file

In [4]:
dist_keys_spl = dist_keys.copy()
for i in range(len(dist_keys_spl)):
    dist_keys_spl[i]=dist_keys_spl[i].split('_')[1]

Creating a list of duplicate districts keys

In [5]:
dupl_lst= ['HP_Hamirpur','UP_Hamirpur','CT_Bilaspur','HP_Bilaspur','UP_Balrampur','CT_Balrampur','MH_Aurangabad','BR_Aurangabad',
                'RJ_Pratapgarh','UP_Pratapgarh']

Creating a dictionary with state name as keys and state code as values for adding district keys to duplicate districts

In [6]:
dat = {'Himachal Pradesh':'HP','Bihar':'BR','Chhattisgarh':'CT','Uttar Pradesh':'UP','Maharashtra':'MH','Rajasthan':'RJ'}

Creating a column to add in the dataframe consisting of District_Keys

In [7]:
dist_names = [0]*len(dist_data)
for i in range(len(dist_data)):
    if dist_data['District'][i] in dist_keys_spl:
        k=dist_keys_spl.index(dist_data['District'][i])
        dist_names[i] = dist_keys[k]

Replacing the duplicate district names list with their appropriate district keys

In [8]:
for i in range(len(dist_data)):
    if dist_names[i] in dupl_lst:
        k=dist_keys.index(dist_names[i])
        dist_names[i] = dat[dist_data['State'][i]]+ '_' +dist_keys_spl[k]

Inserting the District_Keys column into the dataframe

In [9]:
dist_data.insert(3, "District_Key", dist_names, True)

Creating a list of lists containing indices of all occured dates 

In [10]:
indices = [None]*len(date_lst)
for i in range(len(date_lst)):
    indices[i]=[x for x,val in enumerate(dist_data['Date']) if val == date_lst[i]]

Creating a list of lists containing indices of all districts

In [11]:
indices_dist = [None]*len(dist_keys)
for i in range(len(dist_keys)):
    indices_dist[i]=[x for x,val in enumerate(dist_data['District_Key']) if val == dist_keys[i]]

Adding the first date value as it is since in the next code we will start from 1 and on

In [12]:
act_perday = [0]* len(dist_data)
for i in indices[42]:
    act_perday[i]=dist_data['Confirmed'][i]

Creating seperate columns which contain exact number of Confirmed cases for that day where both the columns of the dataframe consisted of cumulative data cases

In [13]:
for i in range(1,len(indices)):
    for j in range(len(indices_dist)):
        k=set(indices[i])&set(indices_dist[j])
        k=list(k)
        m=set(indices[i-1])&set(indices_dist[j])
        m=list(m)
        if len(k)==0 or len(m)==0:
            continue
        act_perday[k[0]]=int(dist_data['Confirmed'][k[0]]) - int(dist_data['Confirmed'][m[0]])

Inserting the created column into the dataframe

In [14]:
dist_data.insert(4, "PerDayConfirmed", act_perday, True)
dist_data

,Date,State,District,District_Key,PerDayConfirmed,Confirmed,Recovered,Deceased,Other,Tested
0,2020-04-26,Andaman and Nicobar Islands,Unknown,0,33,33,11,0,0,NaN
1,2020-04-26,Andhra Pradesh,Anantapur,AP_Anantapur,53,53,14,4,0,NaN
2,2020-04-26,Andhra Pradesh,Chittoor,AP_Chittoor,73,73,13,0,0,NaN
3,2020-04-26,Andhra Pradesh,East Godavari,AP_East Godavari,39,39,12,0,0,NaN
4,2020-04-26,Andhra Pradesh,Guntur,AP_Guntur,214,214,29,8,0,NaN
...,...,...,...,...,...,...,...,...,...,...
319085,2021-09-01,West Bengal,Purba Bardhaman,WB_Purba Bardhaman,0,40613,40161,187,0,NaN
319086,2021-09-01,West Bengal,Purba Medinipur,WB_Purba Medinipur,0,61697,60774,382,0,NaN
319087,2021-09-01,West Bengal,Purulia,WB_Purulia,0,19280,19118,113,0,NaN
319088,2021-09-01,West Bengal,South 24 Parganas,WB_South 24 Parganas,0,97737,95765,1287,0,NaN


Creating list of lists with each inside list corresponding to a date and containing number of daily Confirmed of all districts for that day

In [15]:
confdate_cases=[[0]*len(dist_keys)]*len(date_lst)

Filling number of Confirmed cases values in the confdate_cases list of lists

In [16]:
for i in range(len(date_lst)):
    tmp=[0]*len(dist_keys)
    for j in range(len(dist_keys)):
        k=set(indices[i])&set(indices_dist[j])
        k=list(k)
        if len(k)==0:
            continue
        tmp[j]= dist_data['PerDayConfirmed'][k[0]]
    confdate_cases[i]=tmp

Changing Negative Values to 0

In [18]:
for i in range(len(date_lst)):
    for j in range(len(dist_keys)):
        if confdate_cases[i][j] < 0 :
            confdate_cases[i][j] = 0

Converting confdate_cases to a dataframe and adding Date column

In [19]:
daily_data=pd.DataFrame(confdate_cases, columns = dist_keys)
daily_data.insert(0,"Date",date_lst,True)
daily_data

,Date,AP_Anantapur,AP_Chittoor,AP_East Godavari,AP_Guntur,AP_Krishna,AP_Kurnool,AP_Prakasam,AP_S.P.S. Nellore,AP_Srikakulam,...,WB_Murshidabad,WB_Nadia,WB_North 24 Parganas,WB_Paschim Bardhaman,WB_Paschim Medinipur,WB_Purba Bardhaman,WB_Purba Medinipur,WB_Purulia,WB_South 24 Parganas,WB_Uttar Dinajpur
0,2020-03-15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-03-16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-03-17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-03-18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020-03-19,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,2021-08-10,28,195,98,182,210,12,112,195,41,...,2,46,67,7,36,23,39,1,65,5
514,2021-08-11,51,175,385,222,148,10,98,177,82,...,2,39,89,18,38,16,53,3,63,15
515,2021-08-12,40,233,402,182,144,19,96,225,28,...,9,35,88,9,48,15,46,2,68,6
516,2021-08-13,27,203,304,160,125,20,106,236,91,...,5,41,89,13,43,13,45,4,59,9


Extracting District IDs from the dataframe

In [20]:
dist_names=list(daily_data)
dist_names=dist_names[1:]
dist_names

['AP_Anantapur',
 'AP_Chittoor',
 'AP_East Godavari',
 'AP_Guntur',
 'AP_Krishna',
 'AP_Kurnool',
 'AP_Prakasam',
 'AP_S.P.S. Nellore',
 'AP_Srikakulam',
 'AP_Visakhapatnam',
 'AP_Vizianagaram',
 'AP_West Godavari',
 'AP_Y.S.R. Kadapa',
 'AR_Anjaw',
 'AR_Changlang',
 'AR_East Kameng',
 'AR_East Siang',
 'AR_Kamle',
 'AR_Kra Daadi',
 'AR_Kurung Kumey',
 'AR_Lepa Rada',
 'AR_Lohit',
 'AR_Longding',
 'AR_Lower Dibang Valley',
 'AR_Lower Siang',
 'AR_Lower Subansiri',
 'AR_Namsai',
 'AR_Pakke Kessang',
 'AR_Papum Pare',
 'AR_Shi Yomi',
 'AR_Siang',
 'AR_Tawang',
 'AR_Tirap',
 'AR_Upper Dibang Valley',
 'AR_Upper Siang',
 'AR_Upper Subansiri',
 'AR_West Kameng',
 'AR_West Siang',
 'AS_Baksa',
 'AS_Barpeta',
 'AS_Biswanath',
 'AS_Bongaigaon',
 'AS_Cachar',
 'AS_Charaideo',
 'AS_Chirang',
 'AS_Darrang',
 'AS_Dhemaji',
 'AS_Dhubri',
 'AS_Dibrugarh',
 'AS_Dima Hasao',
 'AS_Goalpara',
 'AS_Golaghat',
 'AS_Hailakandi',
 'AS_Hojai',
 'AS_Jorhat',
 'AS_Kamrup',
 'AS_Kamrup Metropolitan',
 'AS_Karbi

Creating a week list of all Sundays to Saturdays

In [21]:
weekly=[None]*int(len(daily_data)/7)
it = list(range(0,518,7))
for i in it:
    weekly[it.index(i)] = list(range(i,i+7))

Creating a list of lists of size equal to number of districts and each list inside corresponds to one district and is of size of number of weeks and contains the week wise data for that district

In [22]:
week_cases = [[0]*len(weekly)]*len(dist_names)
for i in range(len(dist_names)):
    wc=[0]*len(weekly)
    for j in range(len(weekly)):
        for k in weekly[j]:
            wc[j] = wc[j] + daily_data[dist_names[i]][k]
    week_cases[i]=wc

Creating weekid column for the output file cases-week.csv

In [23]:
week_id = [0]*(len(dist_names)*len(weekly))
lw = len(weekly)
for i in range(len(dist_names)*len(weekly)):
    week_id[i]=(i%lw)+1

Creating districtid column for the output file cases-week.csv

In [24]:
dist_id = [0]*len(week_id)
lw = len(weekly)
ld = len(dist_names)
i=0
for k in range(len(dist_id)):
    dist_id[k]=dist_names[i]
    if int((k+1)/lw) > i:
        i=i+1 

Creating cases column for the output file cases-week.csv

In [25]:
week_casedat = []
for i in week_cases:
    for j in i:
        week_casedat.append(j)

Merging districtid,weekid,cases columns into dataframe

In [26]:
data_final = {'districtid':dist_id, 'weekid':week_id, 'cases':week_casedat}
data_fr = pd.DataFrame(data_final)

Exporting dataframe into cases-week.csv file

In [28]:
data_fr.to_csv("cases-week.csv", index=False)

Creating list containing the corresponding date index of last date (i.e. 14 of a month)

In [29]:
monthly=[-1]
for k in range(len(daily_data)):
    if int(str(daily_data['Date'][k]).split('-')[2]) == 14:
        monthly.append(k-1)

Creating a list of lists of size equal to number of districts and each list inside corresponds to one district and is of size of number of months and contains the month wise data for that district

In [30]:
month_cases = [[0]*(len(monthly)-1)]*len(dist_names)
for i in range(len(dist_names)):
    mc=[0]*(len(monthly)-1)
    for j in range(1,len(monthly)):
        for k in range(monthly[j-1]+1,monthly[j]+1):
            mc[j-1] = mc[j-1] + daily_data[dist_names[i]][k]
    month_cases[i]=mc

Popping the first element from the monthly list

In [31]:
monthly.pop(0)

-1

Creating the monthid column for cases-month.csv

In [32]:
month_id = [0]*(len(dist_names)*len(monthly))
lm = len(monthly)
for i in range(len(dist_names)*len(monthly)):
    month_id[i]=(i%lm)+1

Creating districtid column for file cases-month.csv

In [33]:
distm_id = [0]*len(month_id)
lm = len(monthly)
ld = len(dist_names)
i=0
for k in range(len(distm_id)):
    distm_id[k]=dist_names[i]
    if int((k+1)/lm) > i:
        i=i+1

Creating cases column for file cases-month.csv

In [34]:
month_casedat = []
for i in month_cases:
    for j in i:
        month_casedat.append(j)

Merging the three columns to a dataframe

In [35]:
data_final = {'districtid':distm_id, 'monthid':month_id, 'cases':month_casedat}
data_fr = pd.DataFrame(data_final)
data_fr

,districtid,monthid,cases
0,AP_Anantapur,1,0
1,AP_Anantapur,2,118
2,AP_Anantapur,3,300
3,AP_Anantapur,4,3048
4,AP_Anantapur,5,23868
...,...,...,...
12133,WB_Uttar Dinajpur,13,521
12134,WB_Uttar Dinajpur,14,6139
12135,WB_Uttar Dinajpur,15,5082
12136,WB_Uttar Dinajpur,16,770


Exporting dataframe to a cases-month.csv

In [36]:
data_fr.to_csv('cases-month.csv',index=False)

Creating Overall Column for cases-overall.csv file

In [37]:
overall=[0]*len(dist_names)
for i in range(len(dist_names)):
    oa =0
    for j in range(len(daily_data)):
        oa=oa+daily_data[dist_names[i]][j]
    overall[i]=oa

Creating dataframe with district ids and overall cases

In [38]:
data_final = {'districtid':dist_names, 'overall-cases':overall}
data_fr = pd.DataFrame(data_final)
data_fr

,districtid,overall-cases
0,AP_Anantapur,156771
1,AP_Chittoor,234871
2,AP_East Godavari,282389
3,AP_Guntur,170493
4,AP_Krishna,111599
...,...,...
709,WB_Purba Bardhaman,40280
710,WB_Purba Medinipur,61010
711,WB_Purulia,19213
712,WB_South 24 Parganas,96779


Exporting dataframe to the file cases-overall.csv

In [ ]:
data_fr.to_csv('cases-overall.csv',index=False)